In [1]:
import sys, os, glob
import json
import argparse
import numpy as np
import scipy.io as sio

import open3d as o3d
import copy

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import init
import torch.optim as optim
from torch.optim import lr_scheduler

import smplx
from human_body_prior.tools.model_loader import load_vposer

from utils import GeometryTransformer, BodyParamParser
import time

import os
import pickle
import random
import chamfer_pytorch.dist_chamfer as ext
from torch.autograd import Variable
import trimesh
import json



In [2]:
def body_params_encapsulate(x_body_rec, to_numpy=True, batched=False,nobody=True):
    
    if to_numpy:
        x_body_rec_np = x_body_rec.detach().cpu().numpy()
    else:
        x_body_rec_np = x_body_rec
        
    
    if batched:
        if nobody:
            body_params_batch_rec={}
            body_params_batch_rec['transl'] = x_body_rec_np[:,:3]
            body_params_batch_rec['global_orient'] = x_body_rec_np[:,3:6]
            #body_params_batch_rec['betas'] = x_body_rec_np[:,6:16]
            body_params_batch_rec['body_pose'] = x_body_rec_np[:,6:38]
            body_params_batch_rec['left_hand_pose'] = x_body_rec_np[:,38:50]
            body_params_batch_rec['right_hand_pose'] = x_body_rec_np[:,50:]
        
        else:
            body_params_batch_rec={}
            body_params_batch_rec['transl'] = x_body_rec_np[:,:3]
            body_params_batch_rec['global_orient'] = x_body_rec_np[:,3:6]
            body_params_batch_rec['betas'] = x_body_rec_np[:,6:16]
            body_params_batch_rec['body_pose'] = x_body_rec_np[:,16:48]
            body_params_batch_rec['left_hand_pose'] = x_body_rec_np[:,48:60]
            body_params_batch_rec['right_hand_pose'] = x_body_rec_np[:,60:]

        
        return body_params_batch_rec
    
    else:
        n_batch = x_body_rec_np.shape[0]
        rec_list = []

        for b in range(n_batch):
            body_params_batch_rec={}
            body_params_batch_rec['transl'] = x_body_rec_np[b:b+1,:3]
            body_params_batch_rec['global_orient'] = x_body_rec_np[b:b+1,3:6]
            body_params_batch_rec['betas'] = x_body_rec_np[b:b+1,6:16]
            body_params_batch_rec['body_pose'] = x_body_rec_np[b:b+1,16:48]
            #body_params_batch_rec['left_hand_pose'] = x_body_rec_np[b:b+1,48:60]
            #body_params_batch_rec['right_hand_pose'] = x_body_rec_np[b:b+1,60:]
            rec_list.append(body_params_batch_rec)

        return rec_list




In [4]:
device='cuda'
vposer, _ = load_vposer('./vposer_v1_0', vp_model='snapshot')
vposer=vposer.to(device)

Found Trained Model: ./vposer_v1_0\snapshots\TR00_E096.pt


In [5]:
body_mesh_model_batch = smplx.create('./models', 
                               model_type='smplx',
                               gender='neutral', ext='npz',
                               num_pca_comps=12,
                               create_global_orient=True,
                               create_body_pose=True,
                               create_betas=True,
                               create_left_hand_pose=True,
                               create_right_hand_pose=True,
                               create_expression=True,
                               create_jaw_pose=True,
                               create_leye_pose=True,
                               create_reye_pose=True,
                               create_transl=True,
                               batch_size=60
                               )
body_mesh_model_batch=body_mesh_model_batch.to(device)

body_mesh_model = smplx.create('./models', 
                               model_type='smplx',
                               gender='neutral', ext='npz',
                               num_pca_comps=12,
                               create_global_orient=True,
                               create_body_pose=True,
                               create_betas=True,
                               create_left_hand_pose=True,
                               create_right_hand_pose=True,
                               create_expression=True,
                               create_jaw_pose=True,
                               create_leye_pose=True,
                               create_reye_pose=True,
                               create_transl=True,
                               batch_size=1
                               )
body_mesh_model=body_mesh_model.to(device)

body_mesh_model_input = smplx.create('./models', 
                               model_type='smplx',
                               gender='neutral', ext='npz',
                               num_pca_comps=12,
                               create_global_orient=True,
                               create_body_pose=True,
                               create_betas=True,
                               create_left_hand_pose=True,
                               create_right_hand_pose=True,
                               create_expression=True,
                               create_jaw_pose=True,
                               create_leye_pose=True,
                               create_reye_pose=True,
                               create_transl=True,
                               batch_size=2
                               )

body_mesh_model_input=body_mesh_model_input.to(device)
contact_id_folder='./data/body_segments'
contact_part=['back','gluteus','L_Hand','R_Hand','L_Leg','R_Leg','thighs']
foot_vid, foot_fid = GeometryTransformer.get_contact_id(body_segments_folder=contact_id_folder,
                            contact_body_parts=['L_Leg','R_Leg'])

Sub goal body

In [6]:
from sub_goal import SUBGOAL
ndim=65
cvae=SUBGOAL(n_dim_body=ndim,scene_model_ckpt=False,device=device)
cvae=cvae.to(device)
cvae.load_state_dict(torch.load('saved_model/subgoal.model',map_location=device))

<All keys matched successfully>

In [7]:
#MPH16.ply is a downsampled scene from PROXE dataset
scene_mesh=o3d.io.read_triangle_mesh('demo_data/MPH16.ply')

#camera_ext of MPH16
camera_ext=np.array([[-0.40142276,  0.34250608, -0.84944061,  2.16673488],
                   [ 0.91529181,  0.11642218, -0.38559925,  1.88633201],
                   [-0.03317636, -0.93227435, -0.36022753,  0.736902  ],
                   [ 0.        ,  0.        ,  0.        ,  1.        ]])
camera_ext_inv=np.linalg.inv(camera_ext)

# apply transformation to the scene pointcloud, let the scene and human body in the same area
scene_points=torch.tensor(np.asarray(scene_mesh.transform(camera_ext_inv).vertices),dtype=torch.float32).to(device)

#scene_points=torch.tensor(scene_points,dtype=torch.float32).to(device)
#you can randomly sample the positions or get them from the dataset

start=GeometryTransformer.convert_to_6D_rot(
    torch.tensor([[-0.6111, -0.0384,  2.8900, -2.8275, -0.0543, -0.2241]])).to(device)
sub=GeometryTransformer.convert_to_6D_rot(
    torch.tensor([[-0.3502,  0.1911,  2.1834, -2.8251,  0.0991, -0.7367]])).to(device)
end=GeometryTransformer.convert_to_6D_rot(
    torch.tensor([[-0.6594,  0.2947,  1.2586, -2.8201, -0.1839,  0.5844]])).to(device)

#you can set the body shape parameters
body=torch.zeros([1,10]).to(device)



In [8]:
start_body = cvae.sample(None,scene_points.unsqueeze(0).transpose(1,2),start,body)
start_body_ = GeometryTransformer.convert_to_3D_rot(start_body)
body_param_rec = BodyParamParser.body_params_encapsulate_batch_nobody_hand(start_body_)
body_param_rec['body_pose'] = vposer.decode(body_param_rec['body_pose'], 
                                output_type='aa').view(start_body.shape[0], -1)
body_param_rec['betas']=body.repeat(start_body.shape[0],1)
smplx_output = body_mesh_model(return_verts=True, 
                                      **body_param_rec
                                     )
body_verts_batch = smplx_output.vertices
smplx_faces = body_mesh_model.faces
out_mesh =trimesh.Trimesh(body_verts_batch[0].detach().cpu().numpy(),smplx_faces,process=False)
out_mesh.apply_transform(camera_ext)
out_mesh.export('./demo_data/start.obj')
print('finish')

finish


In [9]:
middle_body = cvae.sample(None,scene_points.unsqueeze(0).transpose(1,2),sub,body)
middle_body_ = GeometryTransformer.convert_to_3D_rot(middle_body)

body_param_rec = BodyParamParser.body_params_encapsulate_batch_nobody_hand(middle_body_)
body_param_rec['body_pose'] = vposer.decode(body_param_rec['body_pose'], 
                                output_type='aa').view(start_body.shape[0], -1)
body_param_rec['betas']=body.repeat(start_body.shape[0],1)
smplx_output = body_mesh_model(return_verts=True, 
                                      **body_param_rec
                                     )
body_verts_batch = smplx_output.vertices
smplx_faces = body_mesh_model.faces
out_mesh =trimesh.Trimesh(body_verts_batch[0].detach().cpu().numpy(),smplx_faces,process=False)
out_mesh.apply_transform(camera_ext)
out_mesh.export('./demo_data/middle.obj')
print('finish')

finish


In [10]:
end_body = cvae.sample(None,scene_points.unsqueeze(0).transpose(1,2),end,body)
end_body_ = GeometryTransformer.convert_to_3D_rot(end_body)

body_param_rec = BodyParamParser.body_params_encapsulate_batch_nobody_hand(end_body_)
body_param_rec['body_pose'] = vposer.decode(body_param_rec['body_pose'], 
                                output_type='aa').view(start_body.shape[0], -1)
body_param_rec['betas']=body.repeat(start_body.shape[0],1)
smplx_output = body_mesh_model(return_verts=True, 
                                      **body_param_rec
                                     )
body_verts_batch = smplx_output.vertices
smplx_faces = body_mesh_model.faces
out_mesh=trimesh.Trimesh(body_verts_batch[0].detach().cpu().numpy(),smplx_faces,process=False)
out_mesh.apply_transform(camera_ext)
out_mesh.export('./demo_data/end.obj')
print('finish')

finish


In [11]:
from route import ROUTENET
from pose_after_route import POSEAFTERROUTE

routenet = ROUTENET(input_dim=9,hid_dim=64,scene_model_ckpt=False,device=device).to(device)
posenet = POSEAFTERROUTE(input_dim=65-9,hid_dim=256,scene_model_ckpt=False,device=device).to(device)

routenet.load_state_dict(torch.load('saved_model/route.model',map_location=device))
posenet.load_state_dict(torch.load('saved_model/pose_after_route.model',map_location=device))
print('finish')

finish


C:\Users\jiash\Anaconda3\lib\site-packages\torch\nn\modules\rnn.py:51: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.4 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


Motion

In [12]:
x1=torch.cat([start_body,middle_body]).unsqueeze(0)
route=routenet(x1[:,:,:9],scene_points.unsqueeze(0).transpose(1,2))
pose=posenet(x1[:,:,9:],scene_points.unsqueeze(0).transpose(1,2),route.view(route.shape[0],-1))

y1=torch.cat([route,pose],dim=2).view(-1,65)
y1_ = GeometryTransformer.convert_to_3D_rot(y1)

body_param_rec = BodyParamParser.body_params_encapsulate_batch_nobody_hand(y1_)
body_param_rec['body_pose'] = vposer.decode(body_param_rec['body_pose'], 
                                output_type='aa').view(y1.shape[0], -1)
body_param_rec['betas']=body.repeat(y1.shape[0],1)
smplx_output = body_mesh_model_batch(return_verts=True, 
                                      **body_param_rec
                                     )
body_verts_batch = smplx_output.vertices
for i in range(60):
    out_mesh =trimesh.Trimesh(body_verts_batch[i].detach().cpu().numpy(),smplx_faces,process=False)
    out_mesh.apply_transform(camera_ext)
    out_mesh.export('./demo_data/'+str(i)+'.obj')

In [13]:
x2=torch.cat([middle_body,end_body]).unsqueeze(0)
route=routenet(x2[:,:,:9],scene_points.unsqueeze(0).transpose(1,2))
pose=posenet(x2[:,:,9:],scene_points.unsqueeze(0).transpose(1,2),route.view(route.shape[0],-1))

y2=torch.cat([route,pose],dim=2).view(-1,65)
y2_ = GeometryTransformer.convert_to_3D_rot(y2)

body_param_rec = BodyParamParser.body_params_encapsulate_batch_nobody_hand(y2_)
body_param_rec['body_pose'] = vposer.decode(body_param_rec['body_pose'], 
                                output_type='aa').view(y2.shape[0], -1)
body_param_rec['betas']=body.repeat(y2.shape[0],1)
smplx_output = body_mesh_model_batch(return_verts=True, 
                                      **body_param_rec
                                     )
body_verts_batch = smplx_output.vertices
for i in range(60):
    out_mesh =trimesh.Trimesh(body_verts_batch[i].detach().cpu().numpy(),smplx_faces,process=False)
    out_mesh.apply_transform(camera_ext)
    out_mesh.export('./demo_data/'+str(i+60)+'.obj')

Optimization

In [14]:
with open('./demo_data/MPH16.json') as f:
        sdf_data = json.load(f)
        grid_min = np.array(sdf_data['min'])
        grid_max = np.array(sdf_data['max'])
        grid_dim = sdf_data['dim']
sdf = np.load('./demo_data/MPH16_sdf.npy').reshape(grid_dim, grid_dim, grid_dim)

In [15]:
s_grid_min = torch.tensor(grid_min, dtype=torch.float32).unsqueeze(0).to(device)
s_grid_max = torch.tensor(grid_max, dtype=torch.float32).unsqueeze(0).to(device)
s_sdf = torch.tensor(sdf, dtype=torch.float32).unsqueeze(0).to(device) 
cam_ext=torch.tensor(camera_ext,dtype=torch.float32).unsqueeze(0).to(device)

In [20]:
from utils_optimize import fitting


contact_id_folder='./data/body_segments'

#connect two short-term motion
x_whole=torch.cat([y1_.detach(),y2_.detach()]) #_,62
start_end=torch.cat([start_body_.detach(),end_body_.detach()]) #_,62


body_mesh_model_batch = smplx.create('./models', 
                               model_type='smplx',
                               gender='neutral', ext='npz',
                               num_pca_comps=12,
                               create_global_orient=True,
                               create_body_pose=True,
                               create_betas=True,
                               create_left_hand_pose=True,
                               create_right_hand_pose=True,
                               create_expression=True,
                               create_jaw_pose=True,
                               create_leye_pose=True,
                               create_reye_pose=True,
                               create_transl=True,
                               batch_size=x_whole.shape[0]
                               )
body_mesh_model_batch=body_mesh_model_batch.to(device)


#first state of the optimization
fittingconfig={'init_lr_h': 0.02,
                'num_iter':20, 
                'contact_part':['back','butt','L_Hand','R_Hand','L_Leg','R_Leg','thighs'],
            }
lossconfig={
        'weight_loss_rec': 0,
        'weight_loss_vposer':0,
        'weight_contact': 2,
        'weight_collision' : 2,
        'weight_motion':0.5,
        'weight_skating':0
    }


y_after = fitting(x_whole, body, cam_ext, 
                    scene_points, s_sdf, s_grid_min, s_grid_max,
                    fittingconfig, lossconfig,start_end,vposer, body_mesh_model_batch, threshold=1.6, contact_id_folder=contact_id_folder, device=device)

fittingconfig={'init_lr_h': 0.02,
                'num_iter':50, 
                'contact_part':['back','butt','L_Hand','R_Hand','L_Leg','R_Leg','thighs'],
                }

lossconfig={
        'weight_loss_rec': 0,
        'weight_loss_vposer':0,
        'weight_contact': 1,
        'weight_collision' : 1,
        'weight_motion':0.5,
        'weight_skating':1
    }
    
y_after = fitting(y_after.detach(), body, cam_ext, 
                    scene_points, s_sdf, s_grid_min, s_grid_max,
                    fittingconfig, lossconfig,start_end,vposer, body_mesh_model_batch, threshold=1.6, contact_id_folder=contact_id_folder, device=device)

torch.Size([1, 10])
torch.Size([120, 62])
['right', 'right', 'right', 'left', 'left', 'left', 'left', 'left', 'left', 'left', 'left', 'left', 'left', 'left', 'left', 'left', 'left', 'left', 'left', 'right', 'right', 'right', 'right', 'right', 'right', 'right', 'right', 'right', 'right', 'right', 'right', 'right', 'left', 'left', 'left', 'left', 'left', 'left', 'left', 'left', 'left', 'left', 'right', 'right', 'right', 'right', 'right', 'left', 'left', 'left', 'left', 'left', 'left', 'left', 'left', 'left', 'left', 'left', 'left', 'left', 'left', 'left', 'left', 'right', 'right', 'right', 'right', 'left', 'left', 'left', 'left', 'left', 'left', 'right', 'right', 'left', 'left', 'left', 'left', 'left', 'left', 'left', 'left', 'left', 'left', 'right', 'right', 'right', 'right', 'right', 'right', 'right', 'right', 'right', 'right', 'right', 'right', 'right', 'right', 'right', 'right', 'right', 'right', 'right', 'right', 'right', 'right', 'right', 'right', 'right', 'right', 'right', 'right'

C:\Users\jiash\Anaconda3\lib\site-packages\torch\nn\functional.py:2693: UserWarning: Default grid_sample and affine_grid behavior will be changed to align_corners=False from 1.4.0. See the documentation of grid_sample for details.
  warnings.warn("Default grid_sample and affine_grid behavior will be changed "


torch.Size([1, 10])
torch.Size([120, 62])
torch.Size([1, 10])
torch.Size([120, 62])
torch.Size([1, 10])
torch.Size([120, 62])
torch.Size([1, 10])
torch.Size([120, 62])
torch.Size([1, 10])
torch.Size([120, 62])
torch.Size([1, 10])
torch.Size([120, 62])
torch.Size([1, 10])
torch.Size([120, 62])
torch.Size([1, 10])
torch.Size([120, 62])
torch.Size([1, 10])
torch.Size([120, 62])
torch.Size([1, 10])
torch.Size([120, 62])
torch.Size([1, 10])
torch.Size([120, 62])
torch.Size([1, 10])
torch.Size([120, 62])
torch.Size([1, 10])
torch.Size([120, 62])
torch.Size([1, 10])
torch.Size([120, 62])
torch.Size([1, 10])
torch.Size([120, 62])
torch.Size([1, 10])
torch.Size([120, 62])
torch.Size([1, 10])
torch.Size([120, 62])
torch.Size([1, 10])
torch.Size([120, 62])
torch.Size([1, 10])
torch.Size([120, 62])
torch.Size([1, 10])
torch.Size([120, 62])
['left', 'left', 'left', 'left', 'left', 'left', 'left', 'left', 'left', 'left', 'left', 'left', 'left', 'left', 'left', 'left', 'right', 'right', 'right', 'rig

In [24]:
body_param_rec = BodyParamParser.body_params_encapsulate_batch_nobody_hand(y_after)
body_param_rec['body_pose'] = vposer.decode(body_param_rec['body_pose'], 
                                output_type='aa').view(y_after.shape[0], -1)
body_param_rec['betas']=body.repeat(y_after.shape[0],1)
smplx_output = body_mesh_model_batch(return_verts=True, 
                                      **body_param_rec
                                     )
body_verts_batch = smplx_output.vertices
for i in range(120):
    out_mesh =trimesh.Trimesh(body_verts_batch[i].detach().cpu().numpy(),smplx_faces,process=False)
    out_mesh.apply_transform(camera_ext)
    out_mesh.export('./demo_data/after'+str(i)+'.obj')